# Here is my thinking in a step-by-step form:
## I'm using IMDB review & sentiment dataset (binary classfication problem)
 1. Clean the text data:
    * lowercase everything
    * remove any weird symbols like "<","/", "\" etc.

2.  Split the data into training and testing sets.
    * I have done by using sklearn (separate file, not included in this one)

3. Load the data using torchtext native methods:
    * Create fields
    * Load data using fields and create TabularDataset

4. Build vocabulary using training dataset, this is where we finally use GloVe.
    * we also make sure that these vectors don't get updated (requires_grad = False)

5. Using iterators for batching the data, for this one we have to again use torchtext native BucketIterator

6. Model, loss function and optimizer are then defined, it looks like the model is working but I don't understand what's going inside.
____

This is what I have issues with:

1. When vocab. is created as far as I understand, we assign a unique code to every unique word which then is being used to map the code to the corresponding vector. Is this what happens?

2. Within every bacth there is $n$ number of reviews, each review has $t$ number of words. Essentially the only dimension that has no variation is the length of the embedding vector (50 in my case). What I fail to understand is what is the shape of our input for the first layer. As I don't undesrstand the ipnut I am lost within the rest of the NN.

3. Why do we need to keep track of $c_t$ and $h_t$ ?
https://pytorch.org/docs/stable/nn.html?highlight=lstm#torch.nn.LSTM

### Anyway, If we could go over the flow of tensors within the network that would really help me.

In [48]:
from torchtext.data import Field, TabularDataset, Iterator, BucketIterator
from torchtext.vocab import GloVe as glove
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 
import torch
import pandas as pd
import numpy as np

In [2]:
folder = '/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets/IMDB_sentiment_analysis'

# this things are required by the torchtext module for effiicent data loading.
TEXT = Field(sequential=True)
LABEL = Field(sequential=False)
fields = [('review', TEXT), ('sentiment', LABEL)] 


train, test = TabularDataset.splits(path=folder,
train = 'train.csv',
test = 'test.csv',
format = 'csv',
skip_header=True,
fields=fields)

In [17]:
# this part builds vocabulary using GLoVe's vectors of length 50.  
TEXT.build_vocab(train,vectors=glove(name='6B', dim=50), max_size=24000, min_freq=10)
LABEL.build_vocab(train)

In [19]:
# this one is a torchtext native iterator that essentially replace dataloader class within standard Pytorch librabry.
train_iter, test_iter = BucketIterator.splits((train, test), batch_size=1) # don't mind batch size of 1, I'm just trying to make it work

In [116]:
class LSTM(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(lstm_out)

In [117]:
model = LSTM(len(TEXT.vocab.stoi), 50, 100)
model.embeddings.weight.data=TEXT.vocab.vectors # the missing link
model.embeddings.weight.requires_grad = False
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad==True], lr=0.0001)

In [119]:
model.train()
for i in range(1):
    for batch_idx, batch in enumerate(train_iter):
        model.zero_grad()
        indices, target = batch.review, batch.sentiment
        out = model(indices)
        print(out)
        break

tensor([[[-0.0173]],

        [[-0.0295]],

        [[-0.0476]],

        [[-0.0445]],

        [[-0.0166]],

        [[-0.0489]],

        [[ 0.0053]],

        [[-0.0242]],

        [[ 0.0244]],

        [[-0.0551]],

        [[-0.0052]],

        [[-0.0532]],

        [[-0.0267]],

        [[-0.0585]],

        [[-0.0022]],

        [[ 0.0205]],

        [[-0.0674]],

        [[-0.0325]],

        [[-0.0362]],

        [[-0.0511]],

        [[-0.0139]],

        [[ 0.0226]],

        [[-0.0203]],

        [[ 0.0133]],

        [[ 0.0444]],

        [[-0.0277]],

        [[ 0.0023]],

        [[-0.0205]],

        [[-0.0623]],

        [[ 0.0248]],

        [[ 0.0094]],

        [[-0.0014]],

        [[-0.0248]],

        [[-0.0419]],

        [[ 0.0376]],

        [[ 0.0357]],

        [[ 0.0054]],

        [[ 0.0212]],

        [[-0.0440]],

        [[-0.0026]],

        [[-0.0316]],

        [[-0.0597]],

        [[ 0.0047]],

        [[-0.0606]],

        [[-0.0285]],

        [[